In [2]:
### Inference on a single image
import os
import json
import torch
import torch.nn as nn
from torchvision.models import EfficientNet_V2_S_Weights, efficientnet_v2_s
from PIL import Image

# --- Configuration ---
MODEL_PATH = 'models/efficientnetv2s/best_model.pth'
# IMAGE_PATH = r'dataset\images\test\Hummus\18.jpg'  # <<<--- Change this to your image file
IMAGE_PATH = r"dataset\images\test\Hummus\65570.jpg"
MAPPING_JSON_PATH = 'class_mapping.json'
NUM_CLASSES = 64

# --- Device Setup ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- Load Class Mapping ---
print(f"Loading class mapping from: {MAPPING_JSON_PATH}")
try:
    with open(MAPPING_JSON_PATH, 'r') as f:
        class_to_idx = json.load(f)
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    if len(idx_to_class) != NUM_CLASSES:
        print(f"Warning: Found {len(idx_to_class)} classes, expected {NUM_CLASSES}")
except Exception as e:
    raise RuntimeError(f"Failed to load class mapping: {e}")

# --- Load Model ---
print("Loading model...")
weights = EfficientNet_V2_S_Weights.IMAGENET1K_V1
model = efficientnet_v2_s(weights=weights)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Sequential(
    nn.Dropout(p=0.37, inplace=True), # Keep dropout
    nn.Linear(num_ftrs, NUM_CLASSES),
)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# --- Preprocessing ---
preprocess = weights.transforms()

# --- Prediction Function ---
def predict_image(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        input_tensor = preprocess(img).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            output = model(input_tensor)
            conf, idx = torch.max(torch.softmax(output, dim=1), 1)
        return idx_to_class.get(idx.item(), "Unknown"), conf.item()
    except Exception as e:
        print(f"Prediction error: {e}")
        return None, None

# --- Inference ---
if not os.path.exists(IMAGE_PATH):
    print(f"Image path does not exist: {IMAGE_PATH}")
else:
    print(f"Predicting image: {IMAGE_PATH}")
    label, confidence = predict_image(IMAGE_PATH)
    if label:
        print(f"-> Predicted Class: {label}")
        print(f"-> Confidence: {confidence:.4f}")

Using device: cuda
Loading class mapping from: class_mapping.json
Loading model...


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_24848\3271592363.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=

Predicting image: dataset\images\test\Hummus\65570.jpg
-> Predicted Class: Cucumber
-> Confidence: 0.3880


#### Yolov8 inference

In [3]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import os
import glob

# Load the YOLOv8x model pretrained on Open Images V7
model = YOLO(r"models\yolov8m-oiv7\weights\best.pt")

# Input and output directories
input_dir = "dataset/images/test/Hummus"
output_dir = "outputs/"
os.makedirs(output_dir, exist_ok=True)

# Supported image formats
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.webp']

# Get all image paths
image_paths = []
for ext in image_extensions:
    image_paths.extend(glob.glob(os.path.join(input_dir, ext)))

print(f"Found {len(image_paths)} image(s) to process.")

# Process each image
for img_path in image_paths:
    print(f"\nProcessing: {img_path}")
    
    # Run prediction
    results = model.predict(source=img_path, save=False, show=False)

    for i, r in enumerate(results):
        # print("\n--- Detection Summary ---")
        # print(f"Boxes: {r.boxes.xyxy}")
        # print(f"Classes: {r.boxes.cls}")
        # print(f"Scores: {r.boxes.conf}")
        # print(f"Class names: {[r.names[int(i)] for i in r.boxes.cls]}")

        # Generate annotated image
        annotated_image = r.plot()

        # Construct output file path
        base_name = os.path.basename(img_path)
        output_path = os.path.join(output_dir, f"annotated_{base_name}")

        # Save image (convert to BGR for OpenCV)
        cv2.imwrite(output_path, annotated_image)
        print(f"Saved annotated image to: {output_path}")

        # Optional: Display the image
        # plt.imshow(annotated_image)
        # plt.axis('off')
        # plt.title(f"Detections for {base_name}")
        # plt.show()

Found 24 image(s) to process.

Processing: dataset/images/test/Hummus\1106758.jpg

image 1/1 c:\Work\Garbage_classification\food-recognition\food-recognition\dataset\images\test\Hummus\1106758.jpg: 480x640 1 Food, 63.8ms
Speed: 5.0ms preprocess, 63.8ms inference, 87.4ms postprocess per image at shape (1, 3, 480, 640)
Saved annotated image to: outputs/annotated_1106758.jpg

Processing: dataset/images/test/Hummus\1317879.jpg

image 1/1 c:\Work\Garbage_classification\food-recognition\food-recognition\dataset\images\test\Hummus\1317879.jpg: 640x640 1 Food, 54.0ms
Speed: 3.4ms preprocess, 54.0ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)
Saved annotated image to: outputs/annotated_1317879.jpg

Processing: dataset/images/test/Hummus\163354.jpg

image 1/1 c:\Work\Garbage_classification\food-recognition\food-recognition\dataset\images\test\Hummus\163354.jpg: 480x640 1 Food, 37.9ms
Speed: 1.9ms preprocess, 37.9ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 